In [1]:
#Initial environement of computations


import matplotlib.pyplot as plt
import numpy as np
from kanon.calendars import Calendar, Date
from kanon.calendars.calendars import float_to_hm, hm_to_float
from kanon.tables import HTable
from kanon.units import Sexagesimal, Historical
from kanon.units.precision import set_precision, TruncatureMode, PrecisionMode
import astropy.units as u


from kanon.tables.htable_reader import (
    DISHAS_REQUEST_URL,
    read_historical,
    read_sexag_array,
    read_intsexag_array,
)
from kanon.tables import HTable
import requests

from kanon.units import Sexagesimal, Historical, IntegerAndSexagesimal, Temporal
from kanon.calendars import*

Cal=julian_ad = Calendar.registry["Julian A.D."]

number_reader = {
    "sexagesimal": read_sexag_array,
    "floating sexagesimal": read_sexag_array,
    "integer and sexagesimal": read_intsexag_array,
    "historical": read_historical,
}

In [9]:
#Initial table set

# Historical target table
Hist_table=HTable.read(590, with_units=False)
Hist_table


"Years, oppositoin",Entries
IntegerAndSexagesimal,Historical
482495 ;,"06s 05 ; 18,35"
491261 ;,"09s 24 ; 26,44"
500027 ;,"01s 13 ; 34,53"
508793 ;,"05s 02 ; 43,02"
517559 ;,"07s 21 ; 10,57"
526325 ;,"11s 10 ; 19,07"
535091 ;,"02s 29 ; 27,16"
543857 ;,"06s 18 ; 35,25"
552623 ;,"10s 07 ; 45,34"


In [10]:
# Formating and initialising the Test tables

Test_recomp=Hist_table.copy()
Delta_recomp=Hist_table.copy()
for i in range(13):
    Test_recomp[i][1]=Historical(Sexagesimal('0;0,0'),2)
    Delta_recomp[i][1]=Historical(Sexagesimal('0;0,0'),2)

In [25]:
#mean motion parameter from the Parisian Alfonsine Tables
#note that the radices for moon lat is not exactly that of the princeps ed of the Parisian Alfonsine Tables
# the Parisian Alfonsine Table one read 3, 34;28, 42, 38, 28
daily_mean_elong=Historical(Sexagesimal("12;11,26,41,37,51,50,39"),7)
radice_elong=Historical(Sexagesimal("3,24;25,49,46,11"),4)
daily_mean_moon_lat=Historical(Sexagesimal("13;13,45,39,22,25,53,45"),7)
radice_moon_lat=Historical(Sexagesimal("3,34;42,42,38,28"),4)
delta_Paris_Toledo=Temporal("0;0,48")
REVO=Historical(Sexagesimal("6,0;0,0"),7)
epoch= Date(Cal, (1, 1, 1))

for i in range(13):
    with set_precision(tmode=TruncatureMode.TRUNC, pmode=5):

        #conversion of date to DN from calendar epoch
        days=int(Hist_table[i][0])-365
        
        #computation of the elongation at date
        elong=(days*daily_mean_elong+radice_elong)%REVO

        #computation of time to conjonction NOT including delta between Paris and Toledo and conversion in DN since epoch
        time=(REVO-elong)/daily_mean_elong
        time=Temporal(time,7)+REVO/(2*daily_mean_elong)
        if time>30:
            time=time-REVO/daily_mean_elong
        else:
            time=time
        time=time.decimal
        time=float(time)+days
        
        #computation of the mean sun (and moon) for that time
        mean_moon_lat=(time*daily_mean_moon_lat+radice_moon_lat)%REVO
        
        
        #filling the Test and Delta tables with the final precision
        with set_precision(tmode=TruncatureMode.ROUND, pmode=2):
            Test_recomp[i][1]=1*mean_moon_lat
            Delta_recomp[i][1]=60*60*(Hist_table[i][1]-Test_recomp[i][1])
Test_recomp

"Years, oppositoin",Entries
IntegerAndSexagesimal,Historical
482495 ;,"06s 05 ; 18,34"
491261 ;,"09s 24 ; 26,43"
500027 ;,"01s 13 ; 34,53"
508793 ;,"05s 02 ; 43,02"
517559 ;,"07s 21 ; 10,57"
526325 ;,"11s 10 ; 19,06"
535091 ;,"02s 29 ; 27,16"
543857 ;,"06s 18 ; 35,25"
552623 ;,"10s 07 ; 43,34"


In [26]:
# residuals visualisation and statistical summaries 1
B=Delta_recomp.to_pandas()
(B.mean().mean(), B.std().std())

(9.538461538461538, nan)

In [27]:
B.astype(int).style.background_gradient(axis=None)

,Entries
"Years, oppositoin",
482495 ;,1
491261 ;,1
500027 ;,0
508793 ;,0
517559 ;,0
526325 ;,1
535091 ;,0
543857 ;,0
552623 ;,120


In [28]:
#csv export of the recomputed table
B=Test_recomp.to_pandas()
compression_opts = dict(method='zip',archive_name='Mean syzygy mean arg lat years opposition.csv')
B.to_csv('Mean syzygy arg lat years opposition.zip', index=False, compression=compression_opts)